In [1]:
import anthropic

client = anthropic.Anthropic()

In [5]:
import os

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

SCOPES = ['https://www.googleapis.com/auth/calendar']

if os.path.exists('./res/token.json'):
    creds = Credentials.from_authorized_user_file('./res/token.json', SCOPES)
else:
    flow = InstalledAppFlow.from_client_secrets_file(
        './res/credentials.json',
        SCOPES
    )

    creds = flow.run_local_server(port=0)

    with open('./res/token.json', 'w') as token:
        token.write(creds.to_json())

service = build('calendar', 'v3', credentials=creds)


In [2]:
messages = []


In [6]:
PROMPT = "2024년 8월 14일 12시부터 1시간 짜리 점심 일정을 생성해주세요."

def create_event(summary, start, end):
  event = {
    "summary": summary,
    "start": {
      "dateTime": start,
      "timeZone": "Asia/Seoul"
    },
    "end": {
      "dateTime": end,
      "timeZone": "Asia/Seoul"
    }
  }

  event = service.events().insert(calendarId='primary', body=event).execute()
  print(f"Event created: %s", (event.get('htmlLink')))
  return event

In [7]:
tools = [
  {
    "name": "create_event",
    "description": "Create new Google Calendar event",
    "input_schema": {
      "type": "object",
      "properties": {
        "summary": {
          "type": "string",
          "description": "Name of Google Calendar event"
        },
        "start": {
          "type": "string",
          "description": "Start date of Google Calendar Event in UTC+9 Time i.e. 2024-08-14T09:00:00"
        },
        "end": {
          "type": "string",
          "description": "End date of Google Calendar Event in UTC+9 Time i.e. 2024-08-14T18:00:00"
        },
               
      },
      "required": ["summary"]
    }
  }
]

In [9]:
response = client.messages.create(
  model="claude-3-5-sonnet-20240620",
  max_tokens=1024,
  tools=tools,
  messages=[{'role': 'user', 'content': PROMPT}],
)

In [10]:
messages.append({"role": "user", "content": PROMPT})

In [11]:
summary = response.content[1].input['summary']
start = response.content[1].input['start']
end = response.content[1].input['end']
event = create_event(summary, start, end)

Event created: %s https://www.google.com/calendar/event?eid=aWU3ZjBpY3Z2bGk5bnJzYWdpOG1xN2dlZGMgbmV2ZXJsaXNoQG0


In [12]:
response = client.messages.create(
  model="claude-3-5-sonnet-20240620",
  max_tokens=1024,
  tools=tools,
  messages=[
    {
      "role": "user",
      "content": PROMPT,
    },
    {
      "role": "assistant",
      "content": [
        {
          "type": "text",
          "text": PROMPT, 
        },
        {
          "type": "tool_use",
          "id": response.content[1].id,
          "name": "create_event",
          "input": {
            "summary": summary,
            "start": start,
            "end": end,
          }
        }
      ],
    },
    {
      "role": "user",
      "content": [
        {
          "type": "tool_result",
          "tool_use_id": response.content[1].id,
          "content": f"Event created: {event.get('htmlLink')}",
        }
      ],
    },
  ],
)
messages.append({"role": "assistant", "content": response.content[0].text})



In [13]:
print(response.content[0].text)

점심 일정이 성공적으로 생성되었습니다. 2024년 8월 14일 오후 12시부터 1시간 동안 "점심 식사" 일정이 Google 캘린더에 추가되었습니다. 일정 링크는 다음과 같습니다: https://www.google.com/calendar/event?eid=aWU3ZjBpY3Z2bGk5bnJzYWdpOG1xN2dlZGMgbmV2ZXJsaXNoQG0

이 일정을 통해 점심 시간을 효율적으로 관리하실 수 있을 것입니다. 다른 도움이 필요하신가요?


In [15]:
new_message = "제가 이전에 어떤 요청을 드렸나요?"

response = client.messages.create(
  model="claude-3-5-sonnet-20240620",
  max_tokens=1024,
  messages=[
    {
      "role": "user",
      "content": new_message,
    },
  ],
)

print(response.content[0].text)



죄송합니다만, 이전 대화 내용을 볼 수 없어서 귀하께서 어떤 요청을 하셨는지 알 수 없습니다. 저는 각각의 대화를 독립적으로 처리하기 때문에 이전 대화 내용을 기억하지 못합니다. 혹시 이전 요청에 대해 더 자세히 말씀해 주시면 도움을 드릴 수 있을 것 같습니다.


In [16]:
new_message = "저의 가장 최근 요청은 어떤 내용이었나요?"

messages.append({"role": "user", "content": new_message})

response = client.messages.create(
  model="claude-3-5-sonnet-20240620",
  max_tokens=1024,
  messages=messages,
)

print(response.content[0].text)



귀하의 가장 최근 요청은 "2024년 8월 14일 12시부터 1시간 짜리 점심 일정을 생성해주세요."였습니다.
